In [12]:
import functions100m as fun100
import functions50m as fun50
import pandas as pd
import tabula

In [21]:
import tabula
import pandas as pd
from datetime import datetime as dt
import csv

def gender_distance_style_category_date_time(lista):
    """
    lista: result of tabula.read_pdf
    Output is (gender, distance, style, category) of the pdf IT NEEDS TO BE USED BEFORE nas_rows
    Functions in which it is being used: pdf_to_df
    """
    df = lista
    date_string = df.iloc[0,0].split(' ')[0]
    time_string = df.iloc[0,0].split(' ')[2]
    names = df.columns
    #names in lowercase
    names = [name.lower() for name in names]
    names = [name.replace('á','a') for name in names]
    names = [name.replace('é','e') for name in names]
    names = [name.replace('í','i') for name in names]
    names = [name.replace('ó','o') for name in names]
    names = [name.replace('ú','u') for name in names]
    # Find string in names which contains 50m, 100m, 200m or 400m
    gender_distance_style = [name for name in names if "50m" in name or "100m" in name or "200m" in name or "400m" in name]
    namessplitpoint = gender_distance_style[0].split('.')
    gender = namessplitpoint[0]
    distance = namessplitpoint[1].split(' ')[1]
    # remove letters in distance
    distance = ''.join([i for i in distance if not i.isalpha()])
    style = namessplitpoint[1].split(' ')[2]

    # last element in names
    namessplitspace = names[-1].split(' ')
    category = namessplitspace[0]

    return gender, distance, style, category, date_string, time_string

def add_columns(df, gender, distance, style, category, date_string, time_string): 
    """"
    add to df 3 columns gender, distance, style, category.
    It should be used together with gender_distance_style_category
    Functions in which it is being used: pdf_to_df
    """
    #add to df 5 columns
    # First gender
    df["gender"] = [gender]*df.shape[0]
    # Second distance
    df["distance"] = [distance]*df.shape[0]
    # Third style
    df["style"] = [style]*df.shape[0]
    # Fourth category
    df["category"] = [category]*df.shape[0]
    # Fifth date
    df["date"] = [date_string]*df.shape[0]
    df["date"] = pd.to_datetime(df["date"], format='%d/%m/%Y')
    # Sixth time
    df["time"] = [time_string]*df.shape[0]
    df["time"] = pd.to_datetime(df["time"], format='%H:%M').dt.time
    return df

def has_whitespace(x):
    return x.count(' ')>0

def list_has_whitespace(x):
    return sum(map(has_whitespace, x))>0

def race_time(df):
    last_column = df.iloc[:, -7].tolist() # it is called last_column because at the beginning it was
    last_column_name = df.columns[-7]
    racetime_dic = dict()
    # if there's a whitespace in any of the rows of the lastcolumn
    if (list_has_whitespace(last_column)):
        #key is the column name
        racetime_dic[last_column_name] = []
        for element in last_column:
            if (has_whitespace(element)):
                race_time = element.split(' ')[0]
                racetime_dic[last_column_name].append(race_time)

            else:
                if(element.count(':')>0):
                    race_time = element
                    racetime_dic[last_column_name].append(race_time)
    else:
        previous_last_column = df.iloc[:,-8].tolist()
        previous_last_column_name = df.columns[-8]
        racetime_dic[previous_last_column_name] = []
        for element in previous_last_column:
            if(element.count(':')>0):
                    race_time = element
                    racetime_dic[last_column_name].append(race_time)
    return racetime_dic
#Datacleaning
def nas_rows(df):
    """
    df: dataframe
    Output: df with rows with only NaN removed and it removes the first two rows
    Functions in which it is being used: pdf_to_df
    """
    # Remove columns with more than 5% NaN
    df.dropna(axis=1, thresh=int(0.8*df.shape[0]), inplace=True)
    # Remove rows with more than 5% NaN
    df.dropna(axis=0, thresh=int(0.8*df.shape[1]), inplace=True)
    # Remove first two rows
    df.drop([0,1], axis=0, inplace=True)
    
    return df

# def column_points(df):
#     """
#     df: dataframe
#     Output: column with more than 20% of hyphens
#     We use this function in points function, in order to find the column we'll called score
#     """
#     for column in df.columns:
#         decimal_count = sum(df[column].apply(lambda x: isinstance(x, str) and x.count('.') == 1 and x.replace('.', '').isdigit()))
#         if decimal_count >= len(df) / 5:
#             return column
#     return  KeyError("No column with more than 20% hyphens found. list_hyphens: {}".format(list_hyphens))

# Example of usage:
# Replace 'your_data_frame' with the name of your DataFrame
# hyphen_columns = find_column_with_half_hyphens(your_data_frame)
# print(hyphen_columns)

# def puntos(df):
#     """
#     df: dataframe
#     Output: list of points and df without the column of points. From now on I will say scores instead of points
#     Functions being used: column_points
#     Functions in which it is being used: evenANDpuntos
#     """

#     # find the column with more than 20% of hyphens
#     column = column_points(df)
#     # extract the forth column starting by the end
#     points = df[column].tolist()
#     # remove elements from puntos which are "-"
#     points = [point for point in points if point != "-"]
#     # remove the column
#     df.drop(column, axis=1, inplace=True)
#     return points, df
def even_odd(df):
    """
    df: dataframe
    Output: df with only even rows and df with only odd rows
    It uses even_odd function
    Functions in which it is being used: evenANDpuntos
    """
    # create a new dataframe with even rows
    even = df.iloc[::2]
    # create a new dataframe with odd rows
    odd = df.iloc[1::2]
    return even, odd

def evenANDpuntos(df):
    """
    df: dataframe
    Output: df with only even rows and a new column for scores
    It uses even_odd and puntos functions
    Functions in which it is being used: pdf_to_df
    """
    even, _ = even_odd(df)
    # score, _ = puntos(df)
    # convert to float
    # score = [float(x) for x in score]
    # add puntos to even
    # even["score"] = score
    # reset index
    even.reset_index(drop=True, inplace=True)
    return even

def remove_accents(input_str):
        s = input_str
        s = s.replace('á','a')
        s = s.replace('é','e')
        s = s.replace('í','i')
        s = s.replace('ó','o')
        s = s.replace('ú','u')
        return s
def make_lowercase(input_str):
    s = input_str
    s = s.lower()
    return s  
def remove_whitespace(input_str):
    s = input_str
    s = s.replace(' ','')
    return s

def find_teams(df, teams_rfen):
    """
    df: dataframe
    Output: list of teams
    It uses even_odd and puntos functions and it uses add_columns function
    Functions in which it is being used: pdf_to_df
    """
    columns = df.columns.tolist()
    for column in columns:
        for element in df[column].tolist():
            if type(element) == str:
                element = make_lowercase(element)
                element = remove_accents(element)
                element = remove_whitespace(element)
                for character in element:
                    # remove digits from element (teams names in some cases are like this: 1. CN Portuense)
                    if character.isdigit():
                        print(element)
                        element = element.replace(character, '')
                if element in teams_rfen["clubes"].tolist():
                    return column
            
def columns_df (df, racetime_dic):
    """
    df: dataframe
    Output: df with column's names "first_surname", "second_surname", "name", "team", "race_time", "gender", "distance", "style", "category", "date", "event_time"
    It uses even_odd and puntos functions and it uses add_columns function
    Functions in which it is being used: pdf_to_df
    """
    df.iloc[:,0] = df.iloc[:,0].str.split('.')
    # df[['first_surname', 'second_surname', 'name']] = df.iloc[:,0].str.split(' ', 2, expand=True)
    # remove the first column
    first_column = df.columns[0]

    # remove rows with nan in any column
    df.dropna(axis=0, how='any', inplace=True)
    # reset index
    df.reset_index(drop=True, inplace=True)

    df[["drop", "full_name"]] = pd.DataFrame(df[first_column].tolist(), index= df.index)
    df.drop(first_column, axis=1, inplace=True)
    df.drop("drop", axis=1, inplace=True)
    # Reset index
    df.reset_index(drop=True, inplace=True)

    # remove , in full_name
    df["full_name"] = df["full_name"].str.replace(',', '')
    # remove first whitespace in full_name
    df["full_name"] = df["full_name"].str.replace(' ', '', 1)
    # split full_name in three columns: firstname, secondname and name
    df['full_name'].str.split()

    # drop rows in df with nan in full_name
    df.dropna(subset=['full_name'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    name_parts = df['full_name'].str.split()

    # Take guiris into account
    for name_part in name_parts:
        if len(name_part) == 2:
            name_part.insert(1, '')
    df['first_surname'] = name_parts.str[0]
    df['first_surname'] = df['first_surname'].str.lower()
    df['second_surname'] = name_parts.str[1]
    df['second_surname'] = df['second_surname'].str.lower()
    df['name'] = name_parts.str[2]
    df['name'] = df['name'].str.lower()
    # remove full_name
    df.drop("full_name", axis=1, inplace=True)

    # first surname first column second surname second column and name third column
    cols = df.columns.tolist()
    cols = cols[-3:] + cols[:-3]
    df = df[cols]

    # reset index
    df.reset_index(drop=True, inplace=True)

    # drop column 5
    df.drop(df.columns[5], axis=1, inplace=True)

    # reset index
    df.reset_index(drop=True, inplace=True)

    # Teams
    file = open("clubs.csv", "r")
    # dataframe from csv file with header
    teams_rfen = pd.DataFrame(csv.reader(file, delimiter=","))
    # make first row as header
    teams_rfen.columns = teams_rfen.iloc[0]
    # remove first row
    teams_rfen = teams_rfen.iloc[1:]
    teams_column_name = find_teams(df, teams_rfen)
    teams = df[teams_column_name]

    # remove the column
    df.drop(teams_column_name, axis=1, inplace=True)
    # remove numbers and first whitespace in third column
    # remove numbers from teams if they exist

    if teams.str.contains('\d').any():
        teams = teams.str.replace('\d+', '')
        # teams = [''.join(char for char in item if not char.isdigit()).strip() for item in teams]

    # remove first whitespace if it exists
    if teams.str.contains(' ').any():
        teams = teams.str.replace(' ', '', 1)
        # teams = [item.lstrip() for item in teams]
    # add teams as the fourth column
    df.insert(loc = 3, column = "team", value = teams)

    # reset index
    df.reset_index(drop=True, inplace=True)
    print(df)
    # drop any column with more than 10% NaN
    df.dropna(axis=1, thresh=int(0.9*df.shape[0]), inplace=True)
    df.reset_index(drop=True, inplace=True)

    # We add race_time
    column = list(racetime_dic.keys())[0]
    values = racetime_dic[column]
    df["race_time"] = values
    # We remove the column in which we extracted race_time
    df.drop(column, axis=1, inplace=True)
    # reset index
    df.reset_index(drop=True, inplace=True)


    #drop columns if they are not 11
    if (len(df.columns) != 11):
        # drop columns named differently from ["first_surname", "second_surname", "name", "team", "race_time", "gender", "distance", "style", "category", "date", "event_time"]
        columns = df.columns.tolist()
        notdropping = ["first_surname", "second_surname", "name", "team", "race_time", "gender", "distance", "style", "category", "date", "event_time"]
        for column in columns:
            if column not in notdropping:
                df.drop(column, axis=1, inplace=True)
                # reset index
                df.reset_index(drop=True, inplace=True)
    df.columns = ["first_surname", "second_surname", "name", "team", "race_time", "gender", "distance", "style", "category", "date", "event_time"]

    # remove integers from team column if they exist
    if df["team"].str.contains('\d').any():
        df["team"] = df["team"].str.replace('\d+', '')
        # df["team"] = [''.join(char for char in item if not char.isdigit()).strip() for item in df["team"]]
    return df

# def delete_columns(df):
#     """
#     df: dataframe
#     Output: df without columns 2 and 4
#     """
#     df.drop(df.columns[2], axis=1, inplace=True)
#     df.drop(df.columns[4], axis=1, inplace=True)
#     return df

def pdf_to_df(pdf): 
    tabu = tabula.read_pdf(pdf, pages='all')
    tabu = tabu[0]
    gender, distance, style, category, date, time = gender_distance_style_category_date_time(tabu)
    tabu = add_columns(tabu, gender=gender, distance=distance, style=style, category=category, date_string=date, time_string=time) #add_columns has dataframe as input
    tabu = nas_rows(tabu)
    # reset index of tabu
    tabu.reset_index(drop=True, inplace=True)
    # tabu = delete_columns(tabu) 
    tabu = evenANDpuntos(tabu)
    racetime_dic = race_time(tabu)
    tabu = columns_df(tabu, racetime_dic)
    return tabu



In [66]:
pdf = "pdfs/trofeo%20portuense%202023/ResultList_41.pdf"
tabu = tabula.read_pdf(pdf, pages='all')
tabu = tabu[0]
print(tabu.iloc[:,-1])

0        Resultados
1     Tiempo Puntos
2     2:18.63 19,00
3     2:18.63 36.55
4     2:19.45 16,00
5     2:19.45 35.68
6     2:24.09 14,00
7     2:24.09 37.71
8     2:24.90 13,00
9     2:24.90 36.80
10    2:25.26 12,00
11    2:25.26 37.79
12    2:25.90 11,00
13    2:25.90 37.41
14    2:30.20 10,00
15    2:30.20 38.80
16     2:31.57 9,00
17    2:31.57 38.70
18     2:32.11 8,00
19    2:32.11 38.60
20     2:34.09 7,00
21    2:34.09 39.86
22     2:40.82 6,00
23    2:40.82 40.83
24     2:52.02 5,00
25    2:52.02 44.24
26     3:00.49 4,00
27    3:00.49 45.84
28     3:08.69 3,00
29    3:08.69 47.98
30     3:10.73 2,00
31    3:10.73 51.38
32     3:17.31 1,00
33    3:17.31 46.03
34        3:21.93 -
35    3:21.93 52.06
36        3:31.69 -
37    3:31.69 53.00
38        3:43.12 -
39    3:43.12 54.97
Name: Absoluto Masculino, dtype: object


In [74]:
tabu = tabula.read_pdf(pdf, pages='all')
tabu = tabu[0]
gender, distance, style, category, date, time = gender_distance_style_category_date_time(tabu)
tabu = add_columns(tabu, gender=gender, distance=distance, style=style, category=category, date_string=date, time_string=time) #add_columns has dataframe as input
tabu = nas_rows(tabu)
# reset index of tabu
tabu.reset_index(drop=True, inplace=True)
# tabu = delete_columns(tabu) 
tabu = evenANDpuntos(tabu)


racetime_dic = race_time(tabu)
df = tabu
df.iloc[:,0] = df.iloc[:,0].str.split('.')
# df[['first_surname', 'second_surname', 'name']] = df.iloc[:,0].str.split(' ', 2, expand=True)
# remove the first column
first_column = df.columns[0]

# remove rows with nan in any column
df.dropna(axis=0, how='any', inplace=True)
# reset index
df.reset_index(drop=True, inplace=True)

df[["drop", "full_name"]] = pd.DataFrame(df[first_column].tolist(), index= df.index)
df.drop(first_column, axis=1, inplace=True)
df.drop("drop", axis=1, inplace=True)
# Reset index
df.reset_index(drop=True, inplace=True)

# remove , in full_name
df["full_name"] = df["full_name"].str.replace(',', '')
# remove first whitespace in full_name
df["full_name"] = df["full_name"].str.replace(' ', '', 1)
# split full_name in three columns: firstname, secondname and name
df['full_name'].str.split()

# drop rows in df with nan in full_name
df.dropna(subset=['full_name'], inplace=True)
df.reset_index(drop=True, inplace=True)

name_parts = df['full_name'].str.split()

# Take guiris into account
for name_part in name_parts:
    if len(name_part) == 2:
        name_part.insert(1, '')
df['first_surname'] = name_parts.str[0]
df['first_surname'] = df['first_surname'].str.lower()
df['second_surname'] = name_parts.str[1]
df['second_surname'] = df['second_surname'].str.lower()
df['name'] = name_parts.str[2]
df['name'] = df['name'].str.lower()
# remove full_name
df.drop("full_name", axis=1, inplace=True)

# first surname first column second surname second column and name third column
cols = df.columns.tolist()
cols = cols[-3:] + cols[:-3]
df = df[cols]

# reset index
df.reset_index(drop=True, inplace=True)

# drop column 5
#df.drop(df.columns[5], axis=1, inplace=True)

# reset index
df.reset_index(drop=True, inplace=True)

# Teams
file = open("clubs.csv", "r")
# dataframe from csv file with header
teams_rfen = pd.DataFrame(csv.reader(file, delimiter=","))
# make first row as header
teams_rfen.columns = teams_rfen.iloc[0]
# remove first row
teams_rfen = teams_rfen.iloc[1:]
teams_column_name = find_teams(df, teams_rfen)
teams = df[teams_column_name]

# remove the column
df.drop(teams_column_name, axis=1, inplace=True)
# remove numbers and first whitespace in third column
# remove numbers from teams if they exist

if teams.str.contains('\d').any():
    teams = teams.str.replace('\d+', '')
    # teams = [''.join(char for char in item if not char.isdigit()).strip() for item in teams]

# remove first whitespace if it exists
if teams.str.contains(' ').any():
    teams = teams.str.replace(' ', '', 1)
    # teams = [item.lstrip() for item in teams]
# add teams as the fourth column
df.insert(loc = 3, column = "team", value = teams)

# reset index
df.reset_index(drop=True, inplace=True)
print(df)
# drop any column with more than 10% NaN
df.dropna(axis=1, thresh=int(0.9*df.shape[0]), inplace=True)
df.reset_index(drop=True, inplace=True)

# We add race_time
column = list(racetime_dic.keys())[0]
values = racetime_dic[column]
df["race_time"] = values
# We remove the column in which we extracted race_time
df.drop(column, axis=1, inplace=True)
# reset index
df.reset_index(drop=True, inplace=True)

# We add event_time
df.rename(columns={"time": "event_time"}, inplace=True)
# reset index
df.reset_index(drop=True, inplace=True)

#drop columns if they are not 11
if (len(df.columns) != 11):
    # drop columns named differently from ["first_surname", "second_surname", "name", "team", "race_time", "gender", "distance", "style", "category", "date", "event_time"]
    columns = df.columns.tolist()
    notdropping = ["first_surname", "second_surname", "name", "team", "race_time", "gender", "distance", "style", "category", "date", "event_time"]
    for column in columns:
        if column not in notdropping:
            df.drop(column, axis=1, inplace=True)
            # reset index
            df.reset_index(drop=True, inplace=True)


print(df.columns)
df.columns = ["first_surname", "second_surname", "name", "team", "race_time", "gender", "distance", "style", "category", "date", "event_time"]

# remove integers from team column if they exist
if df["team"].str.contains('\d').any():
    df["team"] = df["team"].str.replace('\d+', '')

06
6
06
6
08
8
08
8
08
8
09
9
09
9
06
6
09
9
06
6
09
9
02
2
10
0
10
0
09
9
10
0
11

12
2
10
0
   first_surname second_surname       name                       team  \
0          monge         ferruz     marcos                  C.N.Cadiz   
1      belinchon          matos      angel            C.N.Los Barrios   
2           ruiz          manga      oscar             C.D.N.Sanlucar   
3         merino      rodriguez     yeison              C.N.Portuense   
4       gonzalez       ceballos     rafael           C.N.San Fernando   
5      rodriguez          ramos     alvaro           C.N.San Fernando   
6        sanchez         abadía      mario           C.N.Dos Hermanas   
7       pastoril         ibañez      fabio         C.N.Bahia De Cadiz   
8        jimenez          romay     victor              C.N.Portuense   
9       martinez        jimenez     daniel                  C.N.Cadiz   
10      alabarce       manrique     ismael            C.N.Los Barrios   
11        romero           rio

/var/folders/vh/zcb0v6590hq0668t1p4zhgqc0000gn/T/ipykernel_2198/609600086.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(teams_column_name, axis=1, inplace=True)


In [1]:
pdf = "pdfs/trofeo%20portuense%202023/ResultList_41.pdf"

In [4]:
import functions200m as fun200

In [5]:
fun200.pdf_to_df(pdf)

06
6
06
6
08
8
08
8
08
8
09
9
09
9
06
6
09
9
06
6
09
9
02
2
10
0
10
0
09
9
10
0
11

12
2
10
0
   first_surname second_surname       name                       team  \
0          monge         ferruz     marcos                  C.N.Cadiz   
1      belinchon          matos      angel            C.N.Los Barrios   
2           ruiz          manga      oscar             C.D.N.Sanlucar   
3         merino      rodriguez     yeison              C.N.Portuense   
4       gonzalez       ceballos     rafael           C.N.San Fernando   
5      rodriguez          ramos     alvaro           C.N.San Fernando   
6        sanchez         abadía      mario           C.N.Dos Hermanas   
7       pastoril         ibañez      fabio         C.N.Bahia De Cadiz   
8        jimenez          romay     victor              C.N.Portuense   
9       martinez        jimenez     daniel                  C.N.Cadiz   
10      alabarce       manrique     ismael            C.N.Los Barrios   
11        romero           rio

/Users/dankd/Documents/automatization-swimming-data/functions200m.py:295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(teams_column_name, axis=1, inplace=True)


,name,first_surname,second_surname,team,race_time,gender,distance,style,category,date,event_time
0,marcos,monge,ferruz,C.N.Cadiz,2:18.63,masc,200,espalda,absoluto,2023-05-27,19:20:00
1,angel,belinchon,matos,C.N.Los Barrios,2:19.45,masc,200,espalda,absoluto,2023-05-27,19:20:00
2,oscar,ruiz,manga,C.D.N.Sanlucar,2:24.09,masc,200,espalda,absoluto,2023-05-27,19:20:00
3,yeison,merino,rodriguez,C.N.Portuense,2:24.90,masc,200,espalda,absoluto,2023-05-27,19:20:00
4,rafael,gonzalez,ceballos,C.N.San Fernando,2:25.26,masc,200,espalda,absoluto,2023-05-27,19:20:00
5,alvaro,rodriguez,ramos,C.N.San Fernando,2:25.90,masc,200,espalda,absoluto,2023-05-27,19:20:00
6,mario,sanchez,abadía,C.N.Dos Hermanas,2:30.20,masc,200,espalda,absoluto,2023-05-27,19:20:00
7,fabio,pastoril,ibañez,C.N.Bahia De Cadiz,2:31.57,masc,200,espalda,absoluto,2023-05-27,19:20:00
8,victor,jimenez,romay,C.N.Portuense,2:32.11,masc,200,espalda,absoluto,2023-05-27,19:20:00
9,daniel,martinez,jimenez,C.N.Cadiz,2:34.09,masc,200,espalda,absoluto,2023-05-27,19:20:00
